In [9]:
import pandas as pd
import sentier_data_tools as sdt
from datetime import datetime

In [10]:
metadata = sdt.Datapackage(
    name="electricity_markets",
    description="Electricity markets blabla",
    contributors=[
        {
            "title": "Karin Treyer",
            "path": "https://www.psi.ch/en/ta/people/karin-treyer",
            "role": "author",
        },
        {
            "title": "Chris Mutel",
            "path": "https://chris.mutel.org/",
            "role": "wrangler",
        },
    ],
    homepage="https://github.com/polca/premise/tree/master/premise/data/additional_inventories",
).metadata()

In [11]:
columns = ["timestamp", "location", "https://example.com/coal", "https://example.com/gas", "https://example.com/pv"]
timestamp = [datetime(2020, 1, 1, 0, 0, 0), datetime(2020, 1, 2, 0, 0, 0), datetime(2020, 1, 3, 0, 0, 0)]
location = ["PO", "PO", "PO"]
coal = [100, 200, 300]
gas = [50, 100, 150]
pv = [10, 20, 30]
df = pd.DataFrame(list(zip(timestamp, location, coal, gas, pv)), columns=columns)

In [12]:
df

,timestamp,location,https://example.com/coal,https://example.com/gas,https://example.com/pv
0,2020-01-01,PO,100,50,10
1,2020-01-02,PO,200,100,20
2,2020-01-03,PO,300,150,30


In [13]:
UNITS = ["https://example.com/units/datetime", "https://example.com/locationlabel", "https://example.com/units/MW", "https://example.com/units/MW", "https://example.com/units/MW"]

In [14]:
d = sdt.Dataframe(
    name=f"electricity mixes",
    data=df,
    kind=sdt.DataframeKind.BOM,
    product="http://openenergy-platform.org/ontology/oeo/OEO_00000139",
    columns=[{"iri": x, "unit": y} for x, y in zip(df.columns, UNITS)],
    metadata=metadata,
    location="https://sws.geonames.org/6255148/",
    version=1,
    valid_from=datetime(2018, 1, 1),
    valid_to=datetime(2028, 1, 1),
)

In [21]:
d?

Type:        Dataframe
String form: 1
File:        ~/Documents/Coding/sentier_data_tools/sentier_data_tools/local_storage/db.py
Docstring:   <no docstring>

In [18]:
d.save()

1